In [1]:
import sys

sys.path.append("C:/Users/junsu/volatility_arbitrage")

In [2]:
from volatility_arbitrage.pricing_model.interface import HestonParams, MarketModel
from volatility_arbitrage.pricing_model.heston_model import generate_heston_processes
from volatility_arbitrage.pricing_model.weighted_variance_swap import VarianceSwap, SkewSwap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
BDAYS = 252
intraday_samples = 21
length = intraday_samples * 21 * 6
time_delta = 1 / BDAYS / intraday_samples
realized_vol = 0.16
num_path = 100_000

heston_model = HestonParams(kappa=5, mean_of_var=realized_vol**2, vol_of_var=0.5, rho=-0.4)
market_model = MarketModel(
    imp_model=heston_model,
    real_model=heston_model,
    rho_spot_imp_var=heston_model.rho,
    rho_real_var_imp_var=1 - 1e-16,
)
np.random.seed(0)
normal_var = np.random.normal(size=(2, length, num_path))
lr, var = generate_heston_processes(
    var_0=realized_vol**2,
    model_params=heston_model,
    normal_var_1=normal_var[0],
    normal_var_2=normal_var[1],
    num_path=num_path,
    length=length,
    time_delta=time_delta,
)
f = np.exp(lr.cumsum(axis=0))

# Moment

In [9]:
dof = 4 * heston_model.kappa * heston_model.mean_of_var / heston_model.vol_of_var**2
stats = pd.DataFrame(
    columns=pd.MultiIndex.from_product(
        [
            ["Ann. Forward P&L", "Ann. Variance Swap Price", "Volatility Slope"],
            ["Simulated", "Analytic"],
        ]
    ),
    index=pd.Index([1, 10, 21, 63, 126], name="Days"),
)
analytic_mean = 0
var_swap = VarianceSwap(market_model)
skew_swap = SkewSwap(market_model)
for days in stats.index.to_list():
    tau = days / BDAYS
    analytic_var = var_swap.price(imp_var=realized_vol**2, tau=tau)
    analytic_skew = skew_swap.price(imp_var=realized_vol**2, tau=tau)

    simulated_mean = np.mean(f[days * intraday_samples]) - f[0, 0]
    simulated_var = np.mean(var_swap.gamma_pnl(f_t=f[days * intraday_samples], f_0=f[0, 0]))
    simulated_skew = np.mean(skew_swap.gamma_pnl(f_t=f[days * intraday_samples], f_0=f[0, 0]))

    analytic_slope = analytic_skew / analytic_var**1.5
    simulated_slope = simulated_skew / analytic_var**1.5  # Still it should be unbiased

    stats.loc[days] = [
        simulated_mean / tau,
        analytic_mean / tau,
        simulated_var / tau,
        analytic_var / tau,
        simulated_slope,
        analytic_slope,
    ]
display(stats)

Ann. Forward P&L          Ann. Variance Swap Price           \
            Simulated Analytic                Simulated Analytic   
Days                                                               
1                -0.0      0.0                 0.025591   0.0256   
10                0.0      0.0                 0.025522   0.0256   
21                0.0      0.0                 0.025588   0.0256   
63                0.0      0.0                 0.025549   0.0256   
126              -0.0      0.0                 0.025568   0.0256   

     Volatility Slope            
            Simulated  Analytic  
Days                             
1           -0.041756 -0.039102  
10          -0.114216 -0.116363  
21          -0.154383 -0.156956  
63          -0.207687 -0.211735  
126         -0.223745 -0.218915